In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd()) # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")


# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

In [3]:
# Check if the 'theme' column exists
if 'theme' in df.columns:
    # Display unique themes in the 'theme' column
    unique_themes = df['theme'].unique()
    print("Unique themes in the 'theme' column:")
    print(unique_themes)
else:
    print("The 'theme' column does not exist in the dataset.")

Unique themes in the 'theme' column:
['Brussel en de Vlaamse Rand' 'Energie' 'Natuur en Milieu' 'Toerisme'
 'Economie' 'Landbouw' 'Sport' 'Lokale overheden en Binnenlands bestuur'
 'Justitie en Handhaving' 'Media' 'Mobiliteit en Verkeer' 'Gezondheid'
 'Welzijn en Gezin' 'Begroting' 'Wonen' 'Onderwijs en Vorming'
 'Buitenlands beleid' 'Vlaamse administratie'
 'Staatshervorming en Verhoudingen binnen de Belgische federale staat'
 'Onroerend erfgoed' 'Openbare werken' 'Cultuur' 'Financiën'
 'Gelijke kansen' 'Armoedebeleid' 'Sociale economie' 'Jeugdbeleid' 'Werk'
 'Wetenschap en Innovatie' 'Dierenwelzijn' 'Ruimtelijke ordening'
 'Europese instellingen' 'Internationaal ondernemen'
 'Integratie en Inburgering' 'Vlaamse Regering' 'Oekraïnecrisis'
 'Ontwikkelingssamenwerking' 'Vlaams Parlement' 'Taalgebruik']


In [4]:
# Drop unnecessary columns
if "TXT_file_name" in df.columns:
    df = df.drop(columns=["TXT_file_name"])

# Handle missing values
df = df.dropna(subset=["question"])
df["context"].fillna("", inplace=True)

# Clean text
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Replace newlines with spaces
    text = re.sub(r'\b[a-z]\)\s*', ' ', text)  # Remove patterns like 'a)', 'b)', etc., with optional spaces
    text = re.sub(r'\b\d+\.\b', '', text)  # Remove patterns like '1.', '2.', etc.
    text = re.sub(r'\b\d+\)\b', '', text)  # Remove patterns like '1)', '2)', etc.
    text = re.sub(r'\b[i]+[.)]\b', '', text, flags=re.IGNORECASE)  # Remove patterns like 'i.', 'ii.', 'i)', etc.
    text = re.sub(r'\b\d+[.)]\s*', '', text) # Remove numeric list markers like 1., 2. or 1) 2)
    text = re.sub(r'\b[ivxlcdm]+\s*[.)]\s*', '', text, flags=re.IGNORECASE)# Remove roman numerals like i. ii. iii. or i) ii) iii)
    text = re.sub(r'•', '', text)  # Remove bullet symbol
    text = re.sub(r'\[\d+\]', '', text)  # Remove patterns like '[1]', '[2]', etc.
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and trim

    return text
# df["clean_text"] = (df["context"] + " " + df["question"]).apply(clean_text)
df["clean_text"] = (df["question"]).apply(clean_text) 

# Group by 'clean_text' and count unique themes
duplicates_with_diff_themes = df.groupby("clean_text")["theme"].nunique().reset_index()

# Filter rows where the number of unique themes is greater than 1
duplicates_with_diff_themes = duplicates_with_diff_themes[duplicates_with_diff_themes["theme"] > 1]

# Merge back with the original dataframe to get all rows with these 'clean_text'
filtered_df = df[df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]
# Exclude rows with these 'clean_text' from the original dataframe
df = df[~df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]


# Thema-cluster mapping
theme_merge_map = {
    # Bestuur en Beleid
    "Lokale overheden en Binnenlands bestuur": "Bestuur en Beleid",
    "Vlaamse administratie": "Bestuur en Beleid",
    "Staatshervorming en Verhoudingen binnen de Belgische federale staat": "Bestuur en Beleid",
    "Vlaamse Regering": "Bestuur en Beleid",
    "Vlaams Parlement": "Bestuur en Beleid",

    # Mobiliteit en Infrastructuur
    "Mobiliteit en Verkeer": "Mobiliteit en Infrastructuur",
    "Openbare werken": "Mobiliteit en Infrastructuur",
    "Ruimtelijke ordening": "Mobiliteit en Infrastructuur",

    # Economie en Arbeid
    "Werk": "Economie en Arbeid",
    "Economie": "Economie en Arbeid",
    "Sociale economie": "Economie en Arbeid",
    "Internationaal ondernemen": "Economie en Arbeid",

    # Welzijn en Gezondheid
    "Welzijn en Gezin": "Welzijn en Gezondheid",
    "Gezondheid": "Welzijn en Gezondheid",
    "Armoedebeleid": "Welzijn en Gezondheid",

    # Cultuur en Communicatie
    "Cultuur": "Cultuur en Communicatie",
    "Media": "Cultuur en Communicatie",
    "Taalgebruik": "Cultuur en Communicatie",

    # Onderwijs en Samenleving
    "Onderwijs en Vorming": "Onderwijs en Samenleving",
    "Gelijke kansen": "Onderwijs en Samenleving",
    "Jeugdbeleid": "Onderwijs en Samenleving",
    "Integratie en Inburgering": "Onderwijs en Samenleving",

    # Milieu en Landbouw
    "Natuur en Milieu": "Milieu en Landbouw",
    "Landbouw": "Milieu en Landbouw",
    "Dierenwelzijn": "Milieu en Landbouw",

    # Internationaal Beleid
    "Buitenlands beleid": "Internationaal Beleid",
    "Europese instellingen": "Internationaal Beleid",
    "Ontwikkelingssamenwerking": "Internationaal Beleid",
    "Oekraïnecrisis": "Internationaal Beleid",

    # Overige (apart laten tenzij weinig samples)
    "Financiën": "Financiën",
    "Begroting": "Begroting",
    "Wetenschap en Innovatie": "Wetenschap en Innovatie",
    "Toerisme": "Toerisme",
    "Justitie en Handhaving": "Justitie en Handhaving",
    "Brussel en de Vlaamse Rand": "Brussel en de Vlaamse Rand",
    "Sport": "Sport",
    "Onroerend erfgoed": "Onroerend erfgoed",
    "Energie": "Energie",
    "Wonen": "Wonen",
}

# Nieuwe kolom aanmaken met samengevoegde thema's
df["theme_merged"] = df["theme"].map(theme_merge_map).fillna("Onbekend")

# ✅ Now: drop rare themes using original theme names
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 2].index
df = df[df["theme"].isin(valid_themes)]

# ✅ Recompute label encoding AFTER filtering
unique_themes = list(df["theme"].unique())
theme_to_id = {theme: idx for idx, theme in enumerate(unique_themes)}
id_to_theme = {idx: theme for theme, idx in theme_to_id.items()}
df["theme_id"] = df["theme"].map(theme_to_id)

#amount of rows 
print(f"Number of rows after filtering: {len(df)}")

C:\Users\corne\AppData\Local\Temp\ipykernel_24236\162951842.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["context"].fillna("", inplace=True)


Number of rows after filtering: 92502


In [5]:
print(len(df))

92502


In [6]:
# Remove rows where the 'question' column has 5 or fewer words
df = df[df['clean_text'].apply(lambda x: len(str(x).split()) > 9)]

# Remove rows where the 'question' column contains the specific phrase (with flexible matching)
df = df[~df['clean_text'].str.contains(r'\bKan de minister een overzicht geven\b', flags=re.IGNORECASE, na=False)]

# Remove rows where 'clean_text' contains "https:"
df = df[~df['clean_text'].str.contains(r'https:', flags=re.IGNORECASE, na=False)]

In [7]:
print(len(df))

64571


In [8]:
# Save the final DataFrame to an Excel file
output_file_path = os.path.join(data_folder, "Filtered_Data.xlsx")
df.to_excel(output_file_path, index=False)

print(f"DataFrame saved to {output_file_path}")

DataFrame saved to c:\Users\corne\Documents\thesis-question-classification\Data\Filtered_Data.xlsx


In [9]:
text = "b)Zo niet, ziet de minister dit niet als een gemiste kans om de discriminatie tussen de functies van opvoeder en groepschef enerzijds en campusverantwoordelijke anderzijds weg te werken?"

text = re.sub(r'\b[a-z]\)\s*', ' ', text)  # Remove patterns like 'a)', 'b)', etc., with optional spaces
print(text)

 Zo niet, ziet de minister dit niet als een gemiste kans om de discriminatie tussen de functies van opvoeder en groepschef enerzijds en campusverantwoordelijke anderzijds weg te werken?
